# Introduction

This notebook will take a data-driven approach to generating word lists for mental functions that are related to brain circuitry. The overall process is as follows:

1. Cluster brain structures into circuits by PMI-weighted co-occurrences with mental function terms.
2. Identify the mental function terms most highly associated to each circuit over a range of list lengths.
3. Select the list length for each circuit that maximizes word-structure classification performance. 
4. Select the number of circuits that maximizes circuit-function classification performance.

# Load the data

In [1]:
import numpy as np
np.random.seed(42)

import pandas as pd
import sys
sys.path.append("..")
import utilities, ontology

In [2]:
suffix = "lr"

## Brain activation coordinates

In [3]:
act_bin = utilities.load_coordinates()
print("Document N={}, Structure N={}".format(
      act_bin.shape[0], act_bin.shape[1]))

Document N=18155, Structure N=118


## Terms for mental functions

In [4]:
version = 190325
dtm_bin = utilities.load_doc_term_matrix(version=version, binarize=True)

In [5]:
lexicon = utilities.load_lexicon(["cogneuro"])
lexicon = sorted(list(set(lexicon).intersection(dtm_bin.columns)))
len(lexicon)

1683

In [6]:
dtm_bin = dtm_bin[lexicon]
print("Document N={}, Term N={}".format(
      dtm_bin.shape[0], dtm_bin.shape[1]))

Document N=18155, Term N=1683


In [7]:
# Total occurrences of terms in the lexicon
dtm = utilities.load_doc_term_matrix(version=version, binarize=False)
dtm = dtm[lexicon]
np.sum(dtm.values)

4831488

## Document splits

In [8]:
train, val = [[int(pmid.strip()) 
               for pmid in open("../data/splits/{}.txt".format(split))] 
                    for split in ["train", "validation"]]
print("Training N={}, Validation N={}".format(len(train), len(val)))

Training N=12708, Validation N=3631


# Link structures to functions

Links are computed as PMI-weighted co-occurrences across the training set

In [9]:
stm = ontology.load_stm(act_bin.loc[train], dtm_bin.loc[train])
print("Structure N={}, Term N={}".format(stm.shape[0], stm.shape[1]))

Structure N=118, Term N=1634


### Terms most strongly linked to the left amygdala

In [10]:
stm.loc["left_amygdala"].sort_values(ascending=False)[:10]

olfactory_stimulus_transduction      4.286718
auditory_system_function             3.082745
letter_naming_task                   2.900424
eye_puff                             2.494958
face_identification_task             2.440891
pavlovian_conditioning_task          2.035426
emotion_expression_identification    2.018034
waisinformation                      2.018034
social_norm_processing_task          1.984133
offensive_aggression                 1.951343
Name: left_amygdala, dtype: float64

### Structures most strongly linked to *face_identification_task*

In [11]:
stm["face_identification_task"].sort_values(ascending=False)[:10]

right_parahippocampal_gyrus_anterior_division    2.540766
left_frontal_medial_cortex                       2.467574
right_amygdala                                   2.465049
left_parahippocampal_gyrus_anterior_division     2.448795
right_frontal_medial_cortex                      2.442356
left_amygdala                                    2.440891
right_hippocampus                                2.129478
left_temporal_pole                               2.026462
right_temporal_pole                              2.001289
left_cingulate_gyrus_anterior_division           1.582475
Name: face_identification_task, dtype: float64

# Generate the ontology

## 1. Cluster brain structures by functions

In [12]:
import os
from sklearn.cluster import KMeans
from scipy.stats import pointbiserialr

In [13]:
n_circuits = range(2, 51) # Range over which ROC-AUC becomes asymptotic
list_lens = range(5, 51) # Same range as RDoC and the DSM

In [14]:
for k in n_circuits:
    circuit_file = "circuits/circuits_k{:02d}.csv".format(k)
    if not os.path.isfile(circuit_file):
        kmeans = KMeans(n_clusters=k, max_iter=1000, random_state=42)  
        kmeans.fit(stm)
        clust = pd.DataFrame({"STRUCTURE": act_bin.columns, 
                              "CLUSTER": [l+1 for l in list(kmeans.labels_)]})
        clust = clust.sort_values(["CLUSTER", "STRUCTURE"])
        clust.to_csv(circuit_file, index=None)

## 2. Identify associated terms for mental functions

In [15]:
for k in n_circuits:
    circuit_file = "circuits/circuits_k{:02d}.csv".format(k)
    clust = pd.read_csv(circuit_file, index_col=None)
    list_file = "lists/lists_k{:02d}.csv".format(k)
    if not os.path.isfile(list_file):
        lists = pd.DataFrame()
        for i in range(k):
            structures = list(clust.loc[clust["CLUSTER"] == i+1, "STRUCTURE"])
            centroid = np.mean(act_bin.loc[train, structures], axis=1)
            R = pd.Series([pointbiserialr(dtm_bin.loc[train, word], centroid)[0] 
                           for word in dtm_bin.columns], index=dtm_bin.columns)
            R = R[R > 0].sort_values(ascending=False)[:max(list_lens)]
            R = pd.DataFrame({"CLUSTER": [i+1 for l in range(max(list_lens))], 
                              "TOKEN": R.index, "R": R.values})
            lists = lists.append(R)
        lists.to_csv(list_file, index=None)

## 3. Select number of words per domain

Word list lengths were selected by mean ROC-AUC of forward and reverse classifiers for each circuit. The number of circuits will be selected across values of k using the lists of optimized length, this time training classifiers that use all the circuits at that k. All classifiers were optimized over a grid search for regularization strength, penalty, and intercept.

## 4. Select optimal number of domains

### Circuit-level features

In [16]:
import pickle
from sklearn.preprocessing import binarize

In [17]:
directions = ["forward", "reverse"]

In [18]:
fits = {}
for d in directions:
    fits[d] = {}
    for k in n_circuits:
        fit_file = "logistic_regression/sherlock/fits/{}_k{:02d}_{}.p".format(d, k, d)
        fits[d][k] = pickle.load(open(fit_file, "rb"))

/anaconda3/envs/ontol/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.21.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/envs/ontol/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.21.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/envs/ontol/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator OneVsRestClassifier from version 0.21.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [27]:
features = {k: {} for k in n_circuits}
for k in n_circuits:
    domains = range(1, k+1)
    lists, circuits = ontology.load_ontology(k, suffix="_" + suffix)
    function_features = pd.DataFrame(index=dtm_bin.index, columns=domains)
    structure_features = pd.DataFrame(index=act_bin.index, columns=domains)
    for i in domains:
        functions = lists.loc[lists["CLUSTER"] == i, "TOKEN"]
        function_features[i] = dtm_bin[functions].sum(axis=1)
        structures = circuits.loc[circuits["CLUSTER"] == i, "STRUCTURE"]
        structure_features[i] = act_bin[structures].sum(axis=1)
    function_features = pd.DataFrame(utilities.doc_mean_thres(function_features), 
                                     index=dtm_bin.index, columns=domains)
    structure_features = pd.DataFrame(binarize(structure_features), 
                                     index=act_bin.index, columns=domains)
    features[k]["function"] = function_features
    features[k]["structure"] = structure_features

# Name the domains

## Load the lists and circuits of the selected <i>k</i>

In [28]:
op_k = 9

In [29]:
lists, circuits = ontology.load_ontology(op_k, suffix="_" + suffix)

## Select the term with highest degree centrality

Also ensure that names are unique across domains in the clustering solution

In [30]:
k2name = ontology.name_domains(lists, dtm_bin)
k2name

{1: 'HEARING',
 2: 'REWARD',
 3: 'VISION',
 4: 'MANIPULATION',
 5: 'MEMORY',
 6: 'COGNITION',
 7: 'LANGUAGE',
 8: 'EXECUTION',
 9: 'EPISODIC_MEMORY'}

## Sort by predetermined semantic ordering

In [31]:
order = [5,9,2,6,3,4,8,1,7]
k2order = {k: order.index(k)+1 for k in range(1,op_k+1)}
names = [k2name[k] for k in order]
names

['MEMORY',
 'EPISODIC_MEMORY',
 'REWARD',
 'COGNITION',
 'VISION',
 'MANIPULATION',
 'EXECUTION',
 'HEARING',
 'LANGUAGE']

## Export ontology with domain names

### Function term lists

In [37]:
columns = ["ORDER", "CLUSTER", "DOMAIN", "TOKEN", "R", "ROC_AUC"]
lists["ORDER"] = [k2order[k] for k in lists["CLUSTER"]]
lists["DOMAIN"] = [k2name[k] for k in lists["CLUSTER"]]
lists = lists.sort_values(["ORDER", "R"], ascending=[True, False])
lists = lists[columns]
lists.to_csv("lists/lists_data-driven_k09_" + suffix + ".csv", index=None)
lists.head()

,ORDER,CLUSTER,DOMAIN,TOKEN,R,ROC_AUC
52,1,5,MEMORY,memory,0.199402,0.656132
53,1,5,MEMORY,declarative_memory,0.175460,0.656132
54,1,5,MEMORY,episodic_memory,0.172781,0.656132
55,1,5,MEMORY,recognition_memory,0.162780,0.656132
56,1,5,MEMORY,emotion,0.160585,0.656132


### Brain circuits

In [33]:
columns = ["ORDER", "CLUSTER", "DOMAIN", "STRUCTURE"]
circuits["ORDER"] = [k2order[k] for k in circuits["CLUSTER"]]
circuits["DOMAIN"] = [k2name[k] for k in circuits["CLUSTER"]]
circuits = circuits.sort_values(["ORDER", "STRUCTURE"])
circuits = circuits[columns]
circuits.head()

,ORDER,CLUSTER,DOMAIN,STRUCTURE
54,1,5,MEMORY,left_amygdala
55,1,5,MEMORY,left_hippocampus
56,1,5,MEMORY,left_parahippocampal_gyrus_anterior_division
57,1,5,MEMORY,left_parahippocampal_gyrus_posterior_division
58,1,5,MEMORY,left_temporal_fusiform_cortex_anterior_division


In [34]:
circuit_mat = pd.DataFrame(0.0, index=act_bin.columns, columns=names)
for name in names:
    structures = circuits.loc[circuits["DOMAIN"] == name, "STRUCTURE"]
    for structure in structures:
        circuit_mat.loc[structure, name] = 1.0
circuit_mat.to_csv("circuits/clusters_data-driven_k09_" + suffix + ".csv")
circuit_mat.head()

,MEMORY,EPISODIC_MEMORY,REWARD,COGNITION,VISION,MANIPULATION,EXECUTION,HEARING,LANGUAGE
left_frontal_pole,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
left_insular_cortex,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
left_superior_frontal_gyrus,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
left_middle_frontal_gyrus,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
left_inferior_frontal_gyrus_pars_triangularis,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
